# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
Big4='NHL'
# nhl_df.head()
cities.head()


,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [156]:
cities['NHL']

0     Rangers Islanders Devils[note 3]
1                          Kings Ducks
2                       Sharks[note 7]
3                           Blackhawks
4                                Stars
5                             Capitals
6                      Flyers[note 13]
7                               Bruins
8                        Wild[note 16]
9                   Avalanche[note 18]
10                            Panthers
11                             Coyotes
12                           Red Wings
13                         Maple Leafs
14                                   —
15                           [note 25]
16                           Lightning
17                   Penguins[note 28]
18                           [note 32]
19                           [note 34]
20                                   —
21                           [note 39]
22                      Blues[note 43]
23                                   —
24                                   —
25                       

In [29]:
team = cities['NHL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')

team['Metropolitan area']=cities['Metropolitan area']
team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
team= pd.merge(team, cities, how='left', on ='Metropolitan area').iloc[:, 1:4]
team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
team
# team[team['Metropolitan area']=='Columbus']

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
C:\Users\KareemSA\AppData\Local\Temp\ipykernel_15656\3695103896.py:1: SyntaxWarning: invalid escape sequence '\ '
  team = cities['NHL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')


,Metropolitan area,team,Population
0,New York City,Rangers Islanders,20153634
1,Los Angeles,Kings Ducks,13310447
2,San Francisco Bay Area,Sharks,6657982
3,Chicago,Blackhawks,9512999
4,Dallas–Fort Worth,Stars,7233323
5,"Washington, D.C.",Capitals,6131977
6,Philadelphia,Flyers,6070500
7,Boston,Bruins,4794447
8,Minneapolis–Saint Paul,Wild,3551036
9,Denver,Avalanche,2853077


In [30]:
nhl=pd.read_csv("assets/nhl.csv")
nhl = nhl[nhl['year'] == 2018] # Keep only 2018 season
nhl['team'] = nhl['team'].str.replace(r'\*',"", regex=True) # Clean NHL Team Names (Remove "*")
nhl = nhl[['team','W','L']]

nhl
# nhl[nhl['team']=='Blue Jackets']

,team,W,L
0,Atlantic Division,Atlantic Division,Atlantic Division
1,Tampa Bay Lightning,54,23
2,Boston Bruins,50,20
3,Toronto Maple Leafs,49,26
4,Florida Panthers,44,30
5,Detroit Red Wings,30,39
6,Montreal Canadiens,29,40
7,Ottawa Senators,28,43
8,Buffalo Sabres,25,45
9,Metropolitan Division,Metropolitan Division,Metropolitan Division


In [31]:
dropList=[]
for i in range(nhl.shape[0]):
    row=nhl.iloc[i]
    if row['team']==row['W'] and row['L']==row['W']:
        print(row['team'],'will be dropped!')
        dropList.append(i)
nhl=nhl.drop(dropList)

nhl

Atlantic Division will be dropped!
Metropolitan Division will be dropped!
Central Division will be dropped!
Pacific Division will be dropped!


,team,W,L
1,Tampa Bay Lightning,54,23
2,Boston Bruins,50,20
3,Toronto Maple Leafs,49,26
4,Florida Panthers,44,30
5,Detroit Red Wings,30,39
6,Montreal Canadiens,29,40
7,Ottawa Senators,28,43
8,Buffalo Sabres,25,45
10,Washington Capitals,49,26
11,Pittsburgh Penguins,47,29


In [35]:
nhl['team'] = nhl['team'].str.replace('[\w.]* ','', regex=True) #remove until the last word
nhl = nhl.astype({'team': str,'W': int, 'L': int})
nhl['WLRatio'] = nhl['W']/(nhl['W']+nhl['L'])
nhl
# team[team['Metropolitan area']=='Columbus']
# nhl[nhl['team']=='Blue Jackets']

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\KareemSA\AppData\Local\Temp\ipykernel_15656\3957714405.py:1: SyntaxWarning: invalid escape sequence '\w'
  nhl['team'] = nhl['team'].str.replace('[\w.]* ','', regex=True) #remove until the last word


,team,W,L,WLRatio
1,Lightning,54,23,0.701299
2,Bruins,50,20,0.714286
3,Leafs,49,26,0.653333
4,Panthers,44,30,0.594595
5,Wings,30,39,0.434783
6,Canadiens,29,40,0.420290
7,Senators,28,43,0.394366
8,Sabres,25,45,0.357143
10,Capitals,49,26,0.653333
11,Penguins,47,29,0.618421


In [41]:
merge=pd.merge(team,nhl,'outer', on = 'team')
merge=merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})
merge
# merge.loc[merge['Metropolitan area']==' Devils']

C:\Users\KareemSA\AppData\Local\Temp\ipykernel_15656\3903932409.py:2: FutureWarning: The provided callable <function nanmean at 0x000001F1E784BEC0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  merge=merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})
C:\Users\KareemSA\AppData\Local\Temp\ipykernel_15656\3903932409.py:2: FutureWarning: The provided callable <function nanmean at 0x000001F1E784BEC0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  merge=merge.groupby('Metropolitan area').agg({'WLRatio': np.nanmean, 'Population': np.nanmean})


,WLRatio,Population
Metropolitan area,,
Boston,0.714286,4794447.0
Buffalo,0.357143,1132804.0
Calgary,0.513889,1392609.0
Chicago,0.458333,9512999.0
Columbus,NaN,2041520.0
Dallas–Fort Worth,0.567568,7233323.0
Denver,0.589041,2853077.0
Detroit,NaN,4297617.0
Edmonton,0.473684,1321426.0


In [177]:
population_by_region = merge['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from nhl in the same order as cities["Metropolitan area"]
print(win_loss_by_region.dropna())
stats.pearsonr(population_by_region, win_loss_by_region)[0]

Metropolitan area
Boston                    0.714286
Buffalo                   0.357143
Calgary                   0.513889
Chicago                   0.458333
Dallas–Fort Worth         0.567568
Denver                    0.589041
Edmonton                  0.473684
Miami–Fort Lauderdale     0.594595
Minneapolis–Saint Paul    0.633803
Montreal                  0.420290
Nashville                 0.746479
Ottawa                    0.394366
Philadelphia              0.617647
Phoenix                   0.414286
Pittsburgh                0.618421
Raleigh                   0.507042
San Francisco Bay Area    0.625000
St. Louis                 0.578947
Tampa Bay Area            0.701299
Vancouver                 0.436620
Washington, D.C.          0.653333
Winnipeg                  0.722222
Name: W/L%, dtype: float64


np.float64(0.08776391074064943)

In [179]:
def nhl_correlation():
    # YOUR CODE HERE
    # raise NotImplementedError()
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')

    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team= pd.merge(team, cities, how='left', on ='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})

    nhl=pd.read_csv("assets/nhl.csv")
    nhl = nhl[nhl['year'] == 2018] # Keep only 2018 season
    nhl['team'] = nhl['team'].str.replace(r'\*',"", regex=True) # Clean NHL Team Names (Remove "*")
    nhl = nhl[['team','W','L']]

    dropList=[]
    for i in range(nhl.shape[0]):
        row=nhl.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            # print(row['team'],'will be dropped!')
            dropList.append(i)
    nhl=nhl.drop(dropList)

    nhl['team'] = nhl['team'].str.replace('[\w.]* ','', regex=True) #remove until the last word
    nhl = nhl.astype({'team': str,'W': int, 'L': int})
    nhl['W/L%'] = nhl['W']/(nhl['W']+nhl['L'])

    merge=pd.merge(team,nhl,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from nhl in the same order as cities["Metropolitan area"]
    correlation, _=stats.pearsonr(population_by_region, win_loss_by_region)
    return correlation
nhl_correlation()

<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:24: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:24: SyntaxWarning: invalid escape sequence '\w'
C:\Users\KareemSA\AppData\Local\Temp\ipykernel_18860\2491299.py:4: SyntaxWarning: invalid escape sequence '\ '
  team = wikipedia_data_cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
C:\Users\KareemSA\AppData\Local\Temp\ipykernel_18860\2491299.py:24: SyntaxWarning: invalid escape sequence '\w'
  nhl['team'] = nhl['team'].str.replace('[\w.]* ','', regex=True) #remove until the last word
C:\Users\KareemSA\AppData\Local\Temp\ipykernel_18860\2491299.py:29: FutureWarning: The provided callable <function nanmean at 0x000001A6BC01A980> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will

np.float64(nan)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities= cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values